In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
dic_df={}

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        
     
        name=filename.split("-")
        name=name[0]
        dic_df["Data {}".format(name)]=pd.read_csv(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
for name in dic_df:
    print(name)
    df=dic_df[name]
    df.info()
    print("-------------------------------------------------------")

# **Student project:** Predict the CO2 emissions of  Seattle city from the data available ( 2015 or 2016).

This notebook consists of 3 parts

1. Cleanin and feature engineering 
> with a correlation check

2. Prediction of CO2 emissions

>2.1 Comparison of models on the training set

>2.2 Improvement of the Hyperparameters setting of the most efficient model

3. Results analysis on the testing set. 



*I hope you will enjoy reading!!*



In [ ]:
df15=dic_df["Data 2015"]


df15=df15.loc[:,['GHGEmissions(MetricTonsCO2e)','OSEBuildingID', 'BuildingType'
                 , 'PrimaryPropertyType','Neighborhood', 'YearBuilt'
                 ,'NumberofFloors'
                 , 'PropertyGFATotal', 'PropertyGFAParking'
                 ,'PropertyGFABuilding(s)', 'LargestPropertyUseType']]

        




In [ ]:
#count_nan is a function which takes as argument a pd.Serie
 #and retrieve the percentage of np.nan of this series
def count_nan(serie):
    N=serie.shape[0]
    n=serie.isna().sum()
    taux_nan=n/N*100
    return taux_nan

remplissage=[]
for i in df15.columns:
    les_nan=count_nan(df15[i])
    rempl=100-les_nan
    remplissage.append(rempl)
    
#Visualization of the filling rate
(fig, ax) = plt.subplots(figsize=(13, 7))
fig.patch.set_facecolor('#E0E0E0')
fig.patch.set_alpha(0.7)
plt.title("Filling rate \n Dataset 2015")
ax=plt.bar(range(0,len(df15.columns)), remplissage
           ,edgecolor='black',color="#6fd67b")

plt.ylim(40,105)
plt.xlabel("Variables")
plt.ylabel("% of data")
plt.xticks(range(0,len(df15.columns))
           ,df15.columns,rotation=90)
plt.grid()
plt.show(block=True)    


In [ ]:
df15=df15.dropna()
remplissage=[]
for i in df15.columns:
    les_nan=count_nan(df15[i])
    rempl=100-les_nan
    remplissage.append(rempl)
    
#Visualization of the filling rate
# after droping nans 
(fig, ax) = plt.subplots(figsize=(8, 5))
fig.patch.set_facecolor('#E0E0E0')
fig.patch.set_alpha(0.7)
plt.title("Filling rate \n Dataset 2015")
ax=plt.bar(range(0,len(df15.columns)), remplissage
           ,edgecolor='black',color="#6fd67b")

plt.ylim(40,105)
plt.xlabel("Variables")
plt.ylabel("% of data")
plt.xticks(range(0,len(df15.columns))
           ,df15.columns,rotation=90)
plt.grid()
plt.show(block=True)    

# 1. Cleaning and feature engineering

In [ ]:
cat_var=[ 'BuildingType','Neighborhood', 'LargestPropertyUseType']

#This function allows you to view the distribution 
#of CO2 emissions within the classes of a variable.
def visualisation(variable,df):
    the_mean=df["GHGEmissions(MetricTonsCO2e)"].mean()
    fig=plt.figure(figsize=[18,7])
    fig.patch.set_facecolor('#E0E0E0')
    fig.patch.set_alpha(0.7)
    plt.title("C02 emissions distribution by {}".format(variable),size=16)
    sns.boxplot(x=variable, y="GHGEmissions(MetricTonsCO2e)", data=df,color="#cbd1db",width=0.5,showfliers=False,showmeans=True)
    plt.hlines(y=the_mean,xmin=-0.5,xmax=len(df[variable].unique())-0.5,color="#6d788b",ls="--",label="Global mean")

    plt.ylabel(" C02 emissions",size=14)
    plt.xticks(range(0,len(df[variable].unique()))
               ,df[variable].unique(),rotation=90)
    plt.legend()
    plt.grid()
    plt.show()
visualisation('Neighborhood',df15)

In [ ]:
visualisation('BuildingType',df15)

In [ ]:
visualisation('LargestPropertyUseType',df15)

before the transformation of the qualitative variables, I delete some lines with extreme values of co2 emissions
I delete the 5% highest  values and 5% lowest values , 
I consider them as outliers

In [ ]:
borne=round(np.percentile(df15["GHGEmissions(MetricTonsCO2e)"].values, 95),2)
df15=df15.loc[df15["GHGEmissions(MetricTonsCO2e)"]<borne]
borne2=round(np.percentile(df15["GHGEmissions(MetricTonsCO2e)"].values, 5),2)
df15=df15.loc[df15["GHGEmissions(MetricTonsCO2e)"]>borne2]

New visualization after outliers removal

In [ ]:
for v in cat_var:
    visualisation(v,df15)

I identify the categories emitting large quantities of CO2. 

The threshold is set at 2.5 times more than the overall average for the largest Property use type

In [ ]:
#This function will return "True" if the average within the category 
#is greater than x times the overall average. return "False" otherwise
def x_times_more(x,val1,global_mean):
    if val1 > x*global_mean:
        return True
    else:
        return False
#This function will return "True" if the average within the category 
#is lower than x times the overall average. return "False" otherwise    
def x_times_less(x,val1,global_mean):
    if val1 < x*global_mean:
        return True
    else:
        return False 
    
    
#This function returns the list of categories with high and very low CO2 emissions,
#with a threshold (a proportion of the global average)

def large_emission_track(variable, df,tresholdmax,tresholdmin):
    the_mean=df["GHGEmissions(MetricTonsCO2e)"].mean()
    cat=df[variable].unique()
    high_cat=[]
    low_cat=[]
    for c in cat:
        x=df.loc[df[variable]==c]
        m=x["GHGEmissions(MetricTonsCO2e)"].mean()
        if x_times_more(tresholdmax,m,the_mean):
            high_cat.append(c)
        elif x_times_less(tresholdmin,m,the_mean):
            low_cat.append(c)
    return high_cat, low_cat
    

In [ ]:
use_high,use_low=large_emission_track('LargestPropertyUseType', df15,2.5,0.4)
print("Categories with high CO2 emissions: \n",use_high)
print("\n Categories with low CO2 emissions: \n",use_low)

In [ ]:

#Function allowing to encode the level of emission of co2 
#according to the class.
#0 = low emissions
#1 = Average emissions
#2 = high emissions
def encode_usetype(usetype):
    if usetype in use_high:
        return 2
    elif usetype in use_low:
        return 0
    else:
        return 1
#Encoding LargestUseType
df15["largestUseType"]=df15['LargestPropertyUseType'].apply(encode_usetype)
df15=df15.drop(columns=['LargestPropertyUseType'])

In [ ]:
df15.head(1)

In [ ]:
visualisation("PrimaryPropertyType",df15)

same approach for variable "PrimaryPropertyType"

In [ ]:
use_high,use_low=large_emission_track('PrimaryPropertyType', df15,2.5,0.4)
print("Categories with high CO2 emissions: \n",use_high)
print("\n Categories with low CO2 emissions: \n",use_low)

In [ ]:
df15["PropertyType"]=df15['PrimaryPropertyType'].apply(encode_usetype)
df15=df15.drop(columns=['PrimaryPropertyType'])

Same approach for the variable BuildingType with differents treshold

In [ ]:
use_high,use_low=large_emission_track('BuildingType', df15,1.8,0.5)
print("Categories with high CO2 emissions: \n",use_high)
print("\n Categories with low CO2 emissions: \n",use_low)

In [ ]:
df15["BuildingType"]=df15['BuildingType'].apply(encode_usetype)



I use the same type of approach for the "neighborhood" variable but using the median  instead of the mean

In [ ]:
import statistics
def large_emission_track_med(variable, df,tresholdmax,tresholdmin):
    the_mean=statistics.median(df["GHGEmissions(MetricTonsCO2e)"].values)
    cat=df[variable].unique()
    high_cat=[]
    low_cat=[]
    for c in cat:
        x=df.loc[df[variable]==c]
        m=statistics.median(x["GHGEmissions(MetricTonsCO2e)"].values)
        if x_times_more(tresholdmax,m,the_mean):
            high_cat.append(c)
        elif x_times_less(tresholdmin,m,the_mean):
            low_cat.append(c)
    return high_cat, low_cat

In [ ]:
use_high,use_low=large_emission_track('Neighborhood', df15,1.5,0.6)
print("Categories with high CO2 emissions: \n",use_high)
print("\n Categories with low CO2 emissions: \n",use_low)
def encode_downtown(area):
    if area=="DOWNTOWN":
        return 1
    else:
        return 0
df15['Neighborhood']=df15['Neighborhood'].apply(encode_usetype)


I now use the ratio between the parking area and the total area to detect possible outliers.

This new variable can be used for the prediction, and will be added to the work set.

In [ ]:
df15["Parking/building rate"]=df15["PropertyGFAParking"]/df15["PropertyGFATotal"]*100
sns.displot(data=df15, x="Parking/building rate")

We remove the rows where the ratio between the parking area and the total area is greater than 100

In [ ]:
outliers=df15.loc[df15["Parking/building rate"]>100]
df15=df15.loc[df15["Parking/building rate"]<=100]
print(df15.shape)

In [ ]:
print("{} values exceeding 100 for the variable'Parking/building rate'".format(outliers.shape[0]))
outliers.head(5)

Check the linear correlations between the different variables of the dataset

In [ ]:
variables_num=['GHGEmissions(MetricTonsCO2e)', 'BuildingType',
       'Neighborhood', 'YearBuilt', 'NumberofFloors',
       'PropertyGFATotal', 'PropertyGFAParking', 'PropertyGFABuilding(s)',
       'largestUseType', 'PropertyType', 'Parking/building rate']
x=df15.loc[:, variables_num]
# calculate the correlation matrix
corr = x.corr()
lab=x.columns
# plot the heatmap
fig=plt.figure(figsize=[12,9])
fig.patch.set_facecolor('#E0E0E0')
fig.patch.set_alpha(0.7)

plt.title("Linear correlation between features",size=18)
ax=sns.heatmap(corr, vmin=-1, vmax=1,cmap="bwr",
        xticklabels=variables_num,
        yticklabels=variables_num)

Let's check in particular the correlation between the different variables of the dataset and the target variable.

In [ ]:
df=df15.copy()
# to retrieve the coefficients
coef_pearson=[]
coef_spearman=[]
#to retrieve the variables names
labelp=[]
labels=[]
#to retrieve the colors to be assigned to the bars
colorp=[]
colors=[]
bleu="#9fb4ff"
rouge="#ffae9f"

#Variables studied
variables=[  'BuildingType',
       'Neighborhood', 'YearBuilt', 'NumberofFloors',
       'PropertyGFATotal', 'PropertyGFAParking', 'PropertyGFABuilding(s)',
       'largestUseType', 'PropertyType', 'Parking/building rate']

#Variable display threshold
seuil=0.1


for var in variables:
    cp=stats.pearsonr(df['GHGEmissions(MetricTonsCO2e)'].values,df[var])[0]
    cs=stats.spearmanr(df['GHGEmissions(MetricTonsCO2e)'].values,df[var])[0]
   #Pearson coefficients
    if abs(cp)>= seuil:
        coef_pearson.append(cp)
        labelp.append(var)
        if cp>0:
            colorp.append(rouge)
        else:
            colorp.append(bleu)
    #Spearman coefficients                       
    if abs(cs)>= seuil:
        coef_spearman.append(cs)
        labels.append(var)
        if cs>0:
            colors.append(rouge)
        else:
            colors.append(bleu)

In [ ]:
fig=plt.figure(1,figsize=[15,6])
fig.patch.set_facecolor('#E0E0E0')
fig.patch.set_alpha(0.7)
plt.suptitle("correlation with CO2 emissions.",size=16)

# premier barplot des coefficients de corrélation de Pearson
plt.subplot(1,2,1)
plt.title("Pearson correlation")
plt.bar(np.arange(len(labelp))+1,coef_pearson,color=colorp,edgecolor='black')
plt.xticks(np.arange(len(labelp))+1,labelp,rotation=90)
plt.hlines(0,0.5,len(labelp)+0.5,color='black')
plt.ylabel("Coefficient value")
plt.ylim(-0.4,1)
plt.grid()

# 2eme barplot des coefficients de corrélation de Spearman
plt.subplot(1,2,2)
plt.title("Spearman correlation.")
plt.bar(np.arange(len(labels))+1,coef_spearman,color=colors,edgecolor='black')
plt.xticks(np.arange(len(labels))+1,labels,rotation=90)
plt.hlines(0,0.5,len(labels)+0.5,color='black')
plt.ylim(-0.4,1)
plt.grid()

In [ ]:
# Function returning a sample of n rows
def get_sample(N,df):
    n =df.shape[0]
    p =N/n
    sample=df.sample(frac=p, replace=True)
    print("A {} rows sample  has been extract.".format(N))
    return sample


# 2. PREDICTION OF CO2 EMISSIONS .

In [ ]:
sample=get_sample(4000,df15)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

y=sample["GHGEmissions(MetricTonsCO2e)"]
X=sample.drop(columns=['GHGEmissions(MetricTonsCO2e)', 'OSEBuildingID'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


# 2.1 Comparison of models on the training set

In [ ]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import *
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import  make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
#function returning dictionary of model performances on the dataset received as argument
#We work with the r2 scores obtained after a cross validation, with 10 splits
def lets_try(train, y):
    results = {}
    ss=StandardScaler()
    scaled_train=ss.fit_transform(train)
    
   
    def test_model(clf):
        cv = KFold(n_splits=10, shuffle=True, random_state=45)
        r2 = make_scorer(r2_score)
        r2_val_score = cross_val_score(clf, train, y, cv=cv, scoring=r2)
        scores = r2_val_score
        return scores
    
    #for the model which needed standardized data 
    def test_model_scaler(clf):
        cv = KFold(n_splits=10, shuffle=True, random_state=45)
        r2 = make_scorer(r2_score)
        r2_val_score = cross_val_score(clf, scaled_train, y, cv=cv, scoring=r2)
        scores = r2_val_score
        return scores
    
    clf = LinearRegression()
    results["Linear"] = test_model_scaler(clf)
    print("Linear done")
    
    clf = Ridge()
    results["Ridge"] = test_model_scaler(clf)
    print("Ridge done")

    clf = BayesianRidge()
    results["Bayesian Ridge"] = test_model(clf)
    print("Bayesian Ridge done")

    clf = Lasso(alpha=1e-4)
    results["Lasso"] = test_model_scaler(clf)
    print("Lasso done.")

    clf = BaggingRegressor()
    results["Bagging"] = test_model(clf)
    print("Baggind done")

    clf = RandomForestRegressor()
    results["RandomForest"] = test_model(clf)
    print("Random forest done")
   

    clf = AdaBoostRegressor()
    results["AdaBoost"] = test_model(clf)
    print("Adaboost done")

    clf =SVR(kernel='rbf',gamma='auto',C=8)
    results["SVM RBF"] = test_model_scaler(clf)
    print("SVM rbf done")

   
    return results 

In [ ]:
dico_results=lets_try(X_train,y_train)


In [ ]:
fig=plt.figure(figsize=[10,10])
fig.patch.set_facecolor('#E0E0E0')
fig.patch.set_alpha(0.7)
plt.title("Distribution of Cross-validation score  on the trainning set.",size=16)
plt.boxplot(dico_results.values(),labels=dico_results.keys(),showmeans=True)
plt.ylabel("Scores CV \n R2",size=14)
plt.ylim(0,1)
plt.grid()



the random forest model seems to be the most efficient.

**Can we improve the setting of the random forest?**

#  2.2.  Improve the setting .

**Random forest:**
Now we will try to improve the setting of the Max_depth and max_features parameters.

we will use the average of the cross validation scores of the training dataset with 6 splits

In [ ]:
def test_model_mean(clf,train,y):
        cv = KFold(n_splits=6, shuffle=True, random_state=45)
        r2 = make_scorer(r2_score)
        r2_val_score = cross_val_score(clf, train, y, cv=cv, scoring=r2)
        scores = r2_val_score.mean()
        return scores
    
# Maximum depth    
params=[2,3,5,10,15,20,25,30,40,50 ,60]
score_train=[]
for p in params:
    model=RandomForestRegressor() 
    model.set_params(max_depth=p)
    score=test_model_mean(model,X_train,y_train)
    score_train.append(score)
    

In [ ]:
#Visualisation
fig=plt.figure(figsize=[12,5])
fig.patch.set_facecolor('#E0E0E0')
fig.patch.set_alpha(0.7)
plt.title("Mean of Cross-validation score  on the trainning set.",size=16)
plt.plot(params,score_train)
plt.grid()
plt.xlabel("Params max depth value")
plt.ylabel("Mean of the CV score")

The score is maximum and stable from **max_depth=15**

In [ ]:

params=[1,2,3,4,5,6,7,8,9,10]
score_train=[]
for p in params:
    model=RandomForestRegressor() 
    model.set_params(max_features=p)
    score=test_model_mean(model,X_train,y_train)
    score_train.append(score)
    


In [ ]:
#visualisation
fig=plt.figure(figsize=[12,5])
fig.patch.set_facecolor('#E0E0E0')
fig.patch.set_alpha(0.7)
plt.title("Mean of Cross-validation score  on the trainning set.",size=16)
plt.plot(params,score_train)
plt.grid()
plt.xlabel("Params max_features value")
plt.ylabel("Mean of the CV score")
    

The score is maximum  for **max_features=15**

let's check the results of the optimized Random forest model

In [ ]:
best_model=RandomForestRegressor(n_estimators=300, max_depth=15,max_features=2)
def test_model(clf,train,y):
        cv = KFold(n_splits=10, shuffle=True, random_state=45)
        r2 = make_scorer(r2_score)
        r2_val_score = cross_val_score(clf, train, y, cv=cv, scoring=r2)
        scores = r2_val_score
        return scores
the_scores=test_model(best_model,X_train,y_train)
best_model.fit(X_train,y_train)
y_pred=best_model.predict(X_test)
validation_score=r2_score(y_test,y_pred)

In [ ]:
fig=plt.figure(figsize=[10,10])
fig.patch.set_facecolor('#E0E0E0')
fig.patch.set_alpha(0.7)
plt.title("Distribution of Cross-validation score  on the trainning set and testing set.",size=16)
plt.boxplot(the_scores,showmeans=True)
plt.scatter(x=[1],y=[validation_score],s=70,c="r",marker="D",label="Testing CV score \n R2={}".format(round(validation_score,2)))
plt.ylabel("Scores CV \n R2",size=14)
plt.xticks([1],["Random forest"])
plt.ylim(0.5,1)
plt.xlim(0.5,1.5)
plt.grid()
plt.legend()

# 3.  Results analysis: Testing Set  

First,I determine the features that impact the prediction the most

In [ ]:
fig=plt.figure(figsize=[10,10])
fig.patch.set_facecolor('#E0E0E0')
fig.patch.set_alpha(0.7)
plt.title("Feature importances of the Random forest.",size=16)
plt.barh(X.columns, best_model.feature_importances_,color="#28a2b4",edgecolor='black')
plt.grid()

The   prediction result depends mainly on the area of the buildings,  the overall area of the property, and the year of construction.

More modestly, the number of floors and the largest use type also impact the result.

**Now i will analyze the prediction error on the test set**

In [ ]:
df_res=pd.DataFrame({"True values": y_test,"Predictions":y_pred})
df_res["Error"]=abs(df_res["True values"]-df_res["Predictions"])/df_res["True values"]*100
df_res.head()
def color_to_use(num):
    if np.isinf(num):
        return "#cc2900"
    elif num> 80:
        return "#cc2900"
    elif num >50:
        return "#dfa61f"
    elif num>30:
        return "#dddf1f"
    elif num>10:
        return "#7cce2e"
    else:
        return "#13a90d"
df_res["color to use"]=df_res["Error"].apply(color_to_use)
df_res.head()

In [ ]:
df_res=df_res.sort_values(by = "Error")
dict_lab={"#cc2900":'+80%','#dfa61f':"between 50% and 80%"
          ,'#dddf1f':"between 30% and 50%"
          ,'#7cce2e':"between 10% and 30%"
          ,'#13a90d':"Less than 10 %"}
fig=plt.figure(figsize=[10,10])
fig.patch.set_facecolor('#E0E0E0')
fig.patch.set_alpha(0.7)
plt.title("Testing set results, colored by the error rate (%).",size=16)
for c in df_res["color to use"].unique():
    d=df_res.loc[df_res["color to use"]==c]
    sns.scatterplot(data=d,x="True values",y="Predictions",color=c,label=dict_lab[c])

plt.xlabel('True Values ',size=13)
plt.ylabel('Predictions ',size=13)
plt.grid(color='#dddddd')



In [ ]:
def regroup(num):
    if num<50:
        return "- 50"
    elif num<150:
        return "50-150"
    elif num<250:
        return "150-250"
    else:
        return "+250"
df_res["regroup"]=df_res["True values"].apply(regroup)
df_res=df_res.sort_values(by = "True values")

In [ ]:
fig=plt.figure(figsize=[12,6])
fig.patch.set_facecolor('#E0E0E0')
fig.patch.set_alpha(0.7)
plt.title("Distribution of the error rate according to the true values on the testing set.",size=16)
sns.countplot(data=df_res,x="regroup",hue="color to use",hue_order=dict_lab.keys(),palette=['#cc2900', '#dfa61f', '#dddf1f', '#7cce2e', '#13a90d'], edgecolor="black")
plt.legend(dict_lab.values())
plt.grid()
plt.xlabel("CO2 emissions (Metric Tones)")

The large prediction errors (in percentage) are mainly present for the small values to be predicted.

For quantities greater than 50 metric tons of CO2, the model is much more precise

In [ ]:
t,p=df_res["True values"].sum(),df_res["Predictions"].sum()
error=abs(t-p)/t*100
print(error)

fig=plt.figure(figsize=[6,8])
fig.patch.set_facecolor('#E0E0E0')
fig.patch.set_alpha(0.7)
plt.title("Sum of CO2 emissions. \n True values VS Predictions",size=16)
plt.bar(x=[1,2],height=[df_res["True values"].sum(),df_res["Predictions"].sum()],color="#28a2b4",width=0.4, edgecolor="black")
plt.xticks([1,2],["True values","Predictions"])
plt.text(1.7,77000,"Error rate ={} %".format(round(error,1)),size=12)
plt.ylabel("Total CO2 emissions")
plt.ylim(50000,80000)
plt.xlim(0.6,2.6)
plt.grid()

If we consider the total quantity of CO2 emissions for the year 2015, the error rate is + 2.2% which is very low. In this context, the model is efficient.